<h2><center>So what are we cooking?</center></h2>

<div style="background-image: url('https://upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Forestryimages_cassava.jpg/1280px-Forestryimages_cassava.jpg'); border-radius:8px">
    <br>
    <center><h1 style="font-size: 40px">Story Time</h1></center>    
    <br>
</div>

<p style="font-size:20px">This is not a story but more like, trying to understand what exactly is it that we are talking about?
    
<h2 style="font-size:27px">The Cassava Plant</h2>

<p style="font-size:20px">Cassava originated in South America and was introduced to Africa in relatively recent times. It is known to be a very drought-tolerant crop with the ability to yield even when planted in poor soils. When cassava was first grown in Africa, it was used for subsidiary purposes though it is now considered to be one of the most important food staple crops on the continent. Its production is moving toward an industrialized system in which plant material is used for a variety of products including starch, flour, and animal feed.</p>

<h2 style="font-size:27px">Cassava Mosaic Disease</h2>
    
<p style="font-size:20px">It has appeared on major news outlets and is devastating for the farmers in the East African Region</p>
<div>
<img src="https://www.pestnet.org/fact_sheets/assets/image/cassava_bacterial_blight_173/cassavabb.jpg" style="width: 40%; border-radius: 8px; float:left;">
<br><br>

<img src="https://videos.files.wordpress.com/0l5aq83P/cassava_bbc_world_tv_gmt_18_sept_2017_hd.original.jpg" style="width: 60%;margin:10px; border-radius: 8px; float:center;">
    
</div>

<br>

<p style="font-size:20px;">Initially following infection of a cassava geminivirus in cassava, systemic symptoms develop. These symptoms include chlorotic mosaic of the leaves, leaf distortion, and stunted growth. Leaf stalks have a characteristic S-shape. Infection can be overcome by the plant especially when a rapid onset of symptoms occurs. A slow onset of disease development usually correlates with death of the plant.</p>

<h2 style="font-size:27px">Cassava Mosaic Disease</h2>

<p style="font-size:20px;">Control strategies for cassava mosaic disease include sanitation and plant resistance. In this case, sanitation means using cuttings from healthy plants to start with a healthy plot and maintaining that healthy plot by identifying unhealthy plants and immediately removing them. This strategy does not protect them from being inoculated by whiteflies, but research shows that the virus is more aggressive in plants infected from contaminated cuttings than by insect vectors. There are also specific varieties that fare better against some viruses than others, so plant resistance is possible.</p>

<h1 style="font-size: 40px; text-align:center">How Can We Help?</h1>

<p style="font-size:20px;">The problem has been discussed above which leaves us with a very important question: How can we help these people?</p>
    
<p style="font-size:20px;">Folks at Tensorflow have a great idea. <a href="https://www.youtube.com/watch?v=NlpS-DhayQA" style="font-size:20px; text-align:left">Check out this video.</a> </p>
    
<img src="https://bigdata.cgiar.org/wp-content/uploads/2019/06/Pic-4-1-1024x785-1024x785.jpg" style="width:54%; border-radius:8px; float:left; margin: 15px">

<p style="font-size: 20px; margin-left:20px">Mostly grown as a food source in Africa, cassava is the third largest source of carbohydrates in the world. In recent times, cassava production has turned from subsistence to commercial production. CMD was first described in 1894 and is now considered one of the most damaging crop viruses in the world.
The Ministry of Agriculture Food Security and Cooperatives of Tanzania recommend uprooting diseased plants once every week by pulling them out by hand. Plants should be carried away from the field and exposed to the sunlight for drying and then burned to kill the viruses.</p>

<p style="font-size:20px">Finally, lets solve this problem by training a model.</p>

# Importing Libraries

In [ ]:
# Installing EfficientNet
!pip install -q efficientnet_pytorch

In [ ]:
# For the CSVs
import pandas as pd

In [ ]:
# For the Model
from fastai.vision.all import *

from efficientnet_pytorch import EfficientNet

In [ ]:
# For acessing the Files
import os

In [ ]:
# Defining the Path:
path = Path('../input/cassava-leaf-disease-classification')

In [ ]:
# Lets take a look at the CSV.
data = pd.read_csv(path/'train.csv')
train = data[~data['image_id'].isin(['1562043567.jpg', '3551135685.jpg', '2252529694.jpg'])]

<p style="font-size:20px">Defining some functions in order to access the files.</p>

In [ ]:
path = Path('../input/cassava-leaf-disease-classification')

def get_x(r):
    return path/'train_images'/r['image_id']

def get_y(r):
    return r['label']

<p style="font-size:20px">Lets finally make a function for a dataloader</p>

In [ ]:
def get_data(size=256, bs=64, data_df=train):
    block = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                      splitter=RandomSplitter(seed=42), 
                      get_x=get_x,
                      get_y=get_y, 
                      item_tfms = RandomResizedCrop(256),
                      batch_tfms = [*aug_transforms(size=size),
                                    Normalize.from_stats(*imagenet_stats)])

    
    return block.dataloaders(data_df, bs=bs)

In [ ]:
dataloader = get_data()

<p style="font-size:20px">Let's take a look these images once again:</p>

In [ ]:
dataloader.show_batch()

## The Model

<p style="font-size:20px">ITS TIME TO GRAB THE MODEL AND TRAIN IT!</p>

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b2')
learn = Learner(dataloader, model, metrics=accuracy)

# learn.lr_find()

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(5,0.019054606556892395)
# The learnering rate above comes from the lr_find function.

In [ ]:
learn.export("./export.pth")

In [ ]:
learn = load_learner("../input/submission10cassava/export (3).pth")

# Submissions

In [ ]:
submission_df = pd.read_csv(path/'sample_submission.csv')

test_data_path = submission_df['image_id'].apply(lambda x: path/'test_images'/x)
tst_dl = learn.dls.test_dl(test_data_path)
predictions = learn.tta(dl = tst_dl, n=10)

submission_df['label'] = np.argmax(predictions[0],axis=1)
submission_df

In [ ]:
submission_df.to_csv('submission.csv',index=False)

<p style="font-size:20px">That's it folks! Thank you for going through the whole thing. I shouldn't be shamelessly asking for an upvote but it might get me a job and save me from defaulting on my student loan. Hahaha. No I am kidding I go to Delhi University, we don't have student debt here.<br><br> But still. <br> Maybe? 😏😏</p>